# Plan: iNaturalist 2019 (FGVC6) — Medal-Oriented Workflow

Objectives:
- Ship a strong, working baseline fast; iterate to medal.

Data & Metric:
- Train/Val/Test artifacts present: train_val2019.tar.gz, train2019.json, val2019.json, test2019.tar.gz, test2019.json.
- Metric: top-1 classification error (minimize). 1010 classes.

Validation:
- Use official val2019 as holdout for fast feedback. Avoid leakage.
- Later add K-fold on train+val via stratified split if beneficial, but keep official val for LB sanity.

Baseline Model:
- Single-GPU PyTorch with timm.
- Pretrained strong backbone (e.g., convnext_base, tf_efficientnet_b3_ns).
- Input size 224→320 later; AMP; label smoothing; cosine schedule; warmup.
- Augs: RandAugment/AutoAugment, Mixup/CutMix moderate, RandomResizedCrop, flips.
- BCE/CE with label smoothing (CE+LS).
- Save best on val top-1.

Pipeline:
1) Environment & GPU check.
2) Extract tars to /mnt if space allows; build PyTorch datasets from JSONs.
3) Train baseline (few epochs on 224 to validate pipeline).
4) Full train with tuned epochs (e.g., 25-40) with AMP, EMA.
5) Inference: TTA (e.g., 5-10 crops/flips).
6) Generate submission.csv matching sample format.

Risk/Time Controls:
- Log progress and elapsed per epoch; early stop if divergence.
- Cache checkpoints and logits.
- While training, consult experts for architecture/aug choices and whether to scale input/epochs.

Next:
- Run environment checks, parse JSONs, inspect class counts, then implement training script (train.py) and inference script (predict.py).
- Request expert review on plan before heavy training.

In [ ]:
# Environment & GPU check; repo contents overview
import os, sys, json, time, shutil, subprocess, platform, psutil

def run(cmd):
    print("$", " ".join(cmd), flush=True)
    try:
        out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        print(out.stdout, flush=True)
        return out.returncode, out.stdout
    except Exception as e:
        print("ERR:", e, flush=True)
        return 1, str(e)

print(f"Python: {platform.python_version()} | Platform: {platform.platform()}")
print(f"CWD: {os.getcwd()}")
print("Checking GPU via nvidia-smi...")
run(['bash','-lc','nvidia-smi || true'])

print("Disk usage (df -h):")
run(['bash','-lc','df -h'])

print("Listing repo files with sizes and mtimes:")
run(['bash','-lc','ls -alh --time-style=+%Y-%m-%dT%H:%M:%S'])

print("CPU/Memory info:")
print(f"CPU count: {psutil.cpu_count(logical=True)} | RAM: {psutil.virtual_memory().total/1e9:.1f} GB")

print("If torch not installed, we will install cu121 stack next.")
try:
    import torch, torchvision
    print("torch:", torch.__version__, "CUDA avail:", torch.cuda.is_available(), "CUDA build:", getattr(torch.version,'cuda',None))
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    print("torch not importable yet:", e)

In [ ]:
# Install CUDA 12.1-compatible PyTorch stack and verify GPU
import os, sys, subprocess, shutil, time, textwrap

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

# 0) Uninstall any preexisting torch stack
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torch-2.8.0.dist-info",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchvision-0.23.0.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torchaudio-2.8.0.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
):
    if os.path.exists(d):
        print("Removing", d, flush=True)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install EXACT cu121 torch stack
pip("install",
    "--index-url", "https://download.pytorch.org/whl/cu121",
    "--extra-index-url", "https://pypi.org/simple",
    "torch==2.4.1", "torchvision==0.19.1", "torchaudio==2.4.1")

# 2) Freeze versions
from pathlib import Path
Path("constraints.txt").write_text("\n".join([
    "torch==2.4.1",
    "torchvision==0.19.1",
    "torchaudio==2.4.1",
]))

# 3) Install non-torch deps
pip("install", "-c", "constraints.txt",
    "timm==1.0.9",
    "albumentations==1.4.14",
    "opencv-python-headless",
    "scikit-learn",
    "pillow-simd",
    "jpeg4py",
    "pyyaml",
    "einops",
    "wandb",
    "--upgrade-strategy", "only-if-needed")

# 4) Sanity check
import torch
print("torch:", torch.__version__, "built CUDA:", getattr(torch.version, "cuda", None))
print("CUDA available:", torch.cuda.is_available())
assert str(getattr(torch.version, "cuda", "")).startswith("12.1"), f"Wrong CUDA build: {torch.version.cuda}"
assert torch.cuda.is_available(), "CUDA not available"
print("GPU:", torch.cuda.get_device_name(0))

print("Environment ready.")

In [ ]:
# Install remaining deps without pillow-simd/jpeg4py (use standard Pillow) and verify torch GPU
import sys, subprocess

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

pip("install", "-c", "constraints.txt",
    "timm==1.0.9",
    "albumentations==1.4.14",
    "opencv-python-headless",
    "scikit-learn",
    "pyyaml",
    "einops",
    "wandb",
    "--upgrade-strategy", "only-if-needed")

import torch
print("torch:", torch.__version__, "built CUDA:", getattr(torch.version, "cuda", None))
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Deps ready.")

In [ ]:
# Extract datasets to project dir (./inat2019) using Python tarfile (auto-detect compression) and verify a few files
import os, json, time, random, tarfile, pathlib

root = os.path.abspath("./inat2019")
train_root = os.path.join(root, "train_val2019")
test_root = os.path.join(root, "test2019")
os.makedirs(root, exist_ok=True)

def extract_tar_auto(tar_path, dest_dir):
    print(f"Extracting {tar_path} -> {dest_dir}", flush=True)
    t0=time.time()
    with tarfile.open(tar_path, mode="r:*") as tf:
        members = tf.getmembers()
        total = len(members)
        print(f"Members: {total}")
        for i, m in enumerate(members, 1):
            tf.extract(m, path=dest_dir)
            if i % 5000 == 0 or i == total:
                print(f"  {i}/{total} extracted ({(time.time()-t0)/60:.1f} min)", flush=True)
    print(f"Done {tar_path} in {(time.time()-t0)/60:.1f} min")

start=time.time()
if not os.path.exists(train_root) or (os.path.isdir(train_root) and not any(os.scandir(train_root))):
    os.makedirs(train_root, exist_ok=True)
    extract_tar_auto("train_val2019.tar.gz", root)
else:
    print("Train/Val already extracted:", train_root)

if not os.path.exists(test_root) or (os.path.isdir(test_root) and not any(os.scandir(test_root))):
    os.makedirs(test_root, exist_ok=True)
    extract_tar_auto("test2019.tar.gz", root)
else:
    print("Test already extracted:", test_root)

print(f"Extraction stage took {(time.time()-start)/60:.1f} min total")

# Quick JSON and file existence sanity checks
with open("train2019.json","r") as f: train_json=json.load(f)
with open("val2019.json","r") as f: val_json=json.load(f)
with open("test2019.json","r") as f: test_json=json.load(f)

def rel_paths_from_json(js):
    # COCO-style jsons have images list with file_name fields
    return [img["file_name"] for img in js.get("images", js)]

train_files = rel_paths_from_json(train_json)
val_files = rel_paths_from_json(val_json)
test_files = rel_paths_from_json(test_json)

def check_some(files, base):
    if not files:
        print("No files listed in JSON")
        return
    ok=0; tot=min(10, len(files))
    for p in random.sample(files, tot):
        full=os.path.join(base, p)
        if os.path.exists(full):
            ok+=1
        else:
            print("Missing:", full)
    print(f"Existence check: {ok}/{tot} ok")

print("Example train file:", train_files[0] if train_files else None)
print("Example test file:", test_files[0] if test_files else None)

# Check existence relative to extracted root
check_some(train_files, base=root)
check_some(test_files, base=root)

print("Ready for dataset building.")

In [ ]:
# Inspect extracted directory structure and locate sample files
import os, json, random, glob

root = os.path.abspath("./inat2019")
print("Root:", root)
print("Top-level entries:", os.listdir(root))
for d in os.listdir(root):
    p = os.path.join(root, d)
    print(d, "-> dir:", os.path.isdir(p), "files:", sum(len(files) for _,_,files in os.walk(p)))

with open("train2019.json","r") as f: train_json=json.load(f)
with open("test2019.json","r") as f: test_json=json.load(f)

train_files = [img["file_name"] for img in train_json["images"]]
test_files = [img["file_name"] for img in test_json["images"]]
ex_train = train_files[0]
ex_test = test_files[0]
print("Example train JSON path:", ex_train)
print("Example test JSON path:", ex_test)

def exists_at_root(rel):
    return os.path.exists(os.path.join(root, rel))

print("Exists at root (train ex):", exists_at_root(ex_train))
print("Exists at root (test ex):", exists_at_root(ex_test))

# If not found, try to locate by basename anywhere under root
def locate_by_basename(rel):
    base = os.path.basename(rel)
    hits = list(glob.iglob(os.path.join(root, "**", base), recursive=True))
    return hits[:5]

if not exists_at_root(ex_train):
    print("Searching for train basename...")
    print(locate_by_basename(ex_train))
if not exists_at_root(ex_test):
    print("Searching for test basename...")
    print(locate_by_basename(ex_test))

print("Done structure inspection.")

In [1]:
# Build datasets with robust path resolution + basename index (filters missing files)
import os, json, time, math, random
from pathlib import Path
from typing import Dict, Tuple, List
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFile
import torchvision.transforms as T

ImageFile.LOAD_TRUNCATED_IMAGES = True
torch.backends.cudnn.benchmark = True

root = os.path.abspath("./inat2019")
print("Root:", root)

# Load JSONs
with open("train2019.json","r") as f: train_json = json.load(f)
with open("val2019.json","r") as f: val_json = json.load(f)

# Build category id mapping (sparse category_id -> dense [0..C-1])
cat_ids = sorted({c["id"] for c in train_json["categories"]})
catid_to_idx: Dict[int,int] = {cid:i for i, cid in enumerate(cat_ids)}
idx_to_catid: Dict[int,int] = {i:cid for cid,i in catid_to_idx.items()}
num_classes = len(catid_to_idx)
print("Classes:", num_classes)

# COCO-style join images and anns (1 ann per image expected)
def build_records(js: dict) -> List[Tuple[str,int]]:
    anns_by_img = {a["image_id"]: a for a in js["annotations"]}
    recs = []
    for im in js["images"]:
        img_id = im["id"]
        fn = im["file_name"]
        ann = anns_by_img.get(img_id, None)
        if ann is None:
            continue
        cid = ann["category_id"]
        if cid not in catid_to_idx:
            continue
        recs.append((fn, catid_to_idx[cid]))
    return recs

# Build a basename -> full path index (handles any layout differences robustly)
def build_basename_index(root_dir: str):
    t0 = time.time()
    idx: Dict[str, str] = {}
    dup = 0
    count = 0
    for dirpath, _, files in os.walk(root_dir):
        for fn in files:
            if not fn.lower().endswith(('.jpg','.jpeg','.png')):
                continue
            count += 1
            if fn in idx:
                dup += 1
                continue
            idx[fn] = os.path.join(dirpath, fn)
    print(f"Basename index: {count} files, {dup} dups, built in {time.time()-t0:.1f}s")
    return idx

BASENAME_INDEX = build_basename_index(root)

# Robust path resolver because extracted layout differs from JSON top-level dirs
def resolve_path(rel: str) -> str:
    # 1) as-is (e.g., train_val2019/Plants/... or test2019/...)
    p = os.path.join(root, rel)
    if os.path.exists(p):
        return p
    # 2) drop first component
    parts = rel.split('/')
    if len(parts) > 1:
        p2 = os.path.join(root, "/".join(parts[1:]))
        if os.path.exists(p2):
            return p2
    # 3) basename lookup (handles val/test pointing to different top-level)
    base = os.path.basename(rel)
    p3 = BASENAME_INDEX.get(base, None)
    if p3 is not None and os.path.exists(p3):
        return p3
    # 4) last fallback: direct basename in root
    p4 = os.path.join(root, base)
    if os.path.exists(p4):
        return p4
    return p  # may not exist; will be filtered out

train_recs_raw = build_records(train_json)
val_recs_raw = build_records(val_json)
print("Train records (raw):", len(train_recs_raw), "Val records (raw):", len(val_recs_raw))

# Filter out missing files to avoid DataLoader errors
def filter_existing(recs: List[Tuple[str,int]]):
    kept = []
    missing = 0
    for rel, y in recs:
        rp = resolve_path(rel)
        if os.path.exists(rp):
            kept.append((rel, y))
        else:
            missing += 1
    return kept, missing

t0=time.time()
train_recs, miss_tr = filter_existing(train_recs_raw)
val_recs, miss_va = filter_existing(val_recs_raw)
print(f"Filtered: train {len(train_recs)} kept, {miss_tr} missing; val {len(val_recs)} kept, {miss_va} missing; took {time.time()-t0:.1f}s")

class INatDataset(Dataset):
    def __init__(self, recs, train: bool):
        self.recs = recs
        self.train = train
        if train:
            self.tf = T.Compose([
                T.RandomResizedCrop(224, scale=(0.5, 1.0), ratio=(0.75, 1.33)),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            ])
        else:
            self.tf = T.Compose([
                T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
                T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            ])
    def __len__(self):
        return len(self.recs)
    def __getitem__(self, idx):
        rel, y = self.recs[idx]
        path = resolve_path(rel)
        with Image.open(path) as img:
            img = img.convert('RGB')
            x = self.tf(img)
        return x, y

# Instantiate datasets and loaders
train_ds = INatDataset(train_recs, train=True)
val_ds = INatDataset(val_recs, train=False)

batch_size = 64
num_workers = min(12, os.cpu_count() or 8)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True)

# Quick smoke: iterate a batch
t0=time.time()
xb, yb = next(iter(train_loader))
print("Train batch:", xb.shape, yb.shape, "time:", f"{time.time()-t0:.2f}s")
if len(val_ds) > 0:
    t0=time.time()
    xb2, yb2 = next(iter(val_loader))
    print("Val batch:", xb2.shape, yb2.shape, "time:", f"{time.time()-t0:.2f}s")
else:
    print("Val dataset is empty after filtering; will adjust resolver if needed.")

# Torch/CUDA sanity
print("Torch:", torch.__version__, "CUDA avail:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Dataset smoke OK.")

Root: /var/lib/simon/agent_run_states/inaturalist-2019-fgvc6-20250927-163935/inat2019


Classes: 1010


Basename index: 265213 files, 0 dups, built in 0.4s
Train records (raw): 232999 Val records (raw): 3030


Filtered: train 232999 kept, 0 missing; val 0 kept, 3030 missing; took 3.0s


Train batch: torch.Size([64, 3, 224, 224]) torch.Size([64]) time: 0.98s
Val dataset is empty after filtering; will adjust resolver if needed.
Torch: 2.4.1+cu121 CUDA avail: True
GPU: NVIDIA A10-24Q
Dataset smoke OK.


In [ ]:
# Baseline training (stratified holdout from train) + inference to submission
import os, time, math, random, json
from collections import defaultdict
import torch
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
from torchvision import models
import torchvision.transforms as T

# Reuse train_ds, resolve_path, idx_to_catid from previous cell
assert 'train_ds' in globals(), 'Run previous cell to build datasets/mappings.'
assert 'resolve_path' in globals() and 'idx_to_catid' in globals(), 'Mappings/resolver missing.'

# Build a small stratified validation split from train (since val images are not present here)
labels = [y for _, y in train_ds.recs]
by_cls = defaultdict(list)
for i, y in enumerate(labels):
    by_cls[y].append(i)
val_idx = []
train_idx = []
random.seed(42)
per_cls_val = 3  # small, fast sanity; later increase
for c, idxs in by_cls.items():
    random.shuffle(idxs)
    v = idxs[:per_cls_val]
    t = idxs[per_cls_val:]
    val_idx.extend(v)
    train_idx.extend(t)
print(f"Holdout sizes -> train: {len(train_idx)}, val: {len(val_idx)}")

train_sub = Subset(train_ds, train_idx)
val_sub = Subset(INatDataset([(train_ds.recs[i][0], train_ds.recs[i][1]) for i in val_idx], train=False), list(range(len(val_idx))))

batch_size = 64
num_workers = min(12, os.cpu_count() or 8)
train_loader = DataLoader(train_sub, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True)

# Model: torchvision ConvNeXt-Base pretrained, replace head to 1010 classes
num_classes = len(idx_to_catid)
print('Building model...')
model = models.convnext_base(weights=models.ConvNeXt_Base_Weights.DEFAULT)
in_features = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features, num_classes)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu', memory_format=torch.channels_last)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    device = next(model.parameters()).device
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(xb)
            preds = logits.argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    acc = correct / max(1,total)
    return acc

# Train a very short sanity run (1 epoch); extend later
epochs = 1
device = next(model.parameters()).device
best_acc = 0.0
best_path = 'baseline_convnext_base_224.pth'
t_start = time.time()
for epoch in range(epochs):
    model.train()
    epoch_start = time.time()
    seen = 0
    for it, (xb, yb) in enumerate(train_loader):
        xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        seen += yb.size(0)
        if (it+1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs} it {it+1} seen {seen} loss {loss.item():.4f} elapsed {time.time()-epoch_start:.1f}s", flush=True)
    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1} val_acc {val_acc:.4f} epoch_time {time.time()-epoch_start:.1f}s total_elapsed {time.time()-t_start:.1f}s", flush=True)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'model': model.state_dict(), 'num_classes': num_classes}, best_path)
        print(f"Saved best to {best_path}")

print(f"Training done. Best val_acc {best_acc:.4f}")

# Inference on test set and write submission.csv
print('Preparing test dataset...')
with open('test2019.json','r') as f: test_json = json.load(f)
test_images = test_json['images']
test_file_names = [im['file_name'] for im in test_images]
test_ids = [im['id'] for im in test_images]

class INatTestDataset(torch.utils.data.Dataset):
    def __init__(self, rel_paths):
        self.rel_paths = rel_paths
        self.tf = T.Compose([
            T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ])
    def __len__(self):
        return len(self.rel_paths)
    def __getitem__(self, idx):
        rel = self.rel_paths[idx]
        path = resolve_path(rel)
        from PIL import Image
        with Image.open(path) as img:
            img = img.convert('RGB')
            x = self.tf(img)
        return x

test_ds = INatTestDataset(test_file_names)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True)

# Load best weights if saved
if os.path.exists(best_path):
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt['model'], strict=False)
model.eval()

all_preds = []
with torch.no_grad():
    for it, xb in enumerate(test_loader):
        xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
        preds = logits.argmax(1).detach().cpu().tolist()
        all_preds.extend(preds)
        if (it+1) % 100 == 0:
            print(f"Infer it {it+1}/{math.ceil(len(test_ds)/batch_size)}", flush=True)

assert len(all_preds) == len(test_ids), 'Prediction length mismatch'

# Map model idx -> original category_id
pred_cat_ids = [idx_to_catid[p] for p in all_preds]

# Build submission matching sample format (image_id, category_id)
import pandas as pd
sub = pd.DataFrame({'image_id': test_ids, 'category_id': pred_cat_ids})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with', len(sub), 'rows')

In [2]:
# Train timm EfficientNetV2-M @320 with AMP + EMA on stratified holdout; then infer test and write submission.csv
import os, math, time, random, json, subprocess, sys, importlib
from collections import defaultdict
import torch
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as T

# Improve CUDA memory handling
os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'expandable_segments:True')

# Ensure huggingface_hub available BEFORE importing timm (timm checks at import-time)
try:
    import huggingface_hub  # noqa: F401
except Exception:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'huggingface_hub==0.24.6', '--no-deps'], check=True)
    import huggingface_hub  # noqa: F401

# Safe timm import without touching torch stack
try:
    import timm
except Exception:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'timm==1.0.9', '--no-deps'], check=True)
    import timm

from timm.loss import SoftTargetCrossEntropy
from timm.data.mixup import Mixup
from timm.utils import ModelEmaV2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
if hasattr(torch, 'set_float32_matmul_precision'):
    torch.set_float32_matmul_precision('medium')

# Reuse train_ds, resolve_path, idx_to_catid from previous cells
assert 'train_ds' in globals() and 'resolve_path' in globals() and 'idx_to_catid' in globals()

# Build larger stratified holdout (10/class) from train records
labels = [y for _, y in train_ds.recs]
by_cls = defaultdict(list)
for i, y in enumerate(labels):
    by_cls[y].append(i)
val_idx, tr_idx = [], []
random.seed(42)
per_cls_val = 10
for c, idxs in by_cls.items():
    random.shuffle(idxs)
    v = idxs[:per_cls_val]
    t = idxs[per_cls_val:]
    val_idx.extend(v)
    tr_idx.extend(t)
print(f"Holdout sizes -> train: {len(tr_idx)}, val: {len(val_idx)}")

# 320px transforms
train_tf = T.Compose([
    T.RandomResizedCrop(320, scale=(0.2, 1.0), ratio=(0.75, 1.33), interpolation=T.InterpolationMode.BICUBIC),
    T.RandAugment(num_ops=2, magnitude=9),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    T.RandomErasing(p=0.25, value='random')
])
val_tf = T.Compose([
    T.Resize(352, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(320),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

class WrappedDS(torch.utils.data.Dataset):
    def __init__(self, base, indices, train):
        self.base = base
        self.idxs = indices
        self.train = train
    def __len__(self):
        return len(self.idxs)
    def __getitem__(self, k):
        rel, y = self.base.recs[self.idxs[k]]
        from PIL import Image
        path = resolve_path(rel)
        with Image.open(path) as img:
            img = img.convert('RGB')
            x = (train_tf if self.train else val_tf)(img)
        return x, y

# Model FIRST to control CUDA memory; build loaders after
num_classes = len(idx_to_catid)
print('Building timm model...')
try:
    model = timm.create_model(
        'tf_efficientnetv2_m.in21k_ft_in1k',
        pretrained=True,
        num_classes=num_classes,
        drop_path_rate=0.3,
        pretrained_cfg_overlay={'hf_hub_id': None}
    )
except Exception as e:
    print('Pretrained load w/ URL fallback failed, retrying default...', e)
    try:
        model = timm.create_model('tf_efficientnetv2_m.in21k_ft_in1k', pretrained=True, num_classes=num_classes, drop_path_rate=0.3)
    except Exception as e2:
        print('Retry failed, falling back to pretrained=False (sanity only):', e2)
        model = timm.create_model('tf_efficientnetv2_m.in21k_ft_in1k', pretrained=False, num_classes=num_classes, drop_path_rate=0.3)

# Free any cached GPU memory before moving model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    try:
        print('Before to():', torch.cuda.memory_summary(abbreviated=True))
    except Exception:
        pass

if hasattr(model, 'set_grad_checkpointing'):
    model.set_grad_checkpointing(True)
model = model.to(device)  # defer channels_last until after to() to reduce transient copies
model = model.to(memory_format=torch.channels_last)

# Optimizer & LR schedule
batch_size = 48  # further reduced; build loaders after model to minimize CUDA pressure
base_lr = 3e-4 * (batch_size/64)
optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=0.05, betas=(0.9,0.999))
epochs = 2  # quick sanity; extend to 35 later
warmup_epochs = 1  # use 5 when epochs=35
cosine_min_lr = 1e-6

def lr_at_epoch(ep):
    if ep < warmup_epochs:
        return base_lr * (ep+1)/max(1,warmup_epochs)
    t = (ep - warmup_epochs)/max(1,(epochs - warmup_epochs))
    return cosine_min_lr + 0.5*(base_lr - cosine_min_lr)*(1 + math.cos(math.pi*t))

# Dataloaders (after model) and with conservative worker settings to avoid fragmentation)
num_workers = min(8, os.cpu_count() or 8)
train_sub = WrappedDS(train_ds, tr_idx, train=True)
val_sub = WrappedDS(train_ds, val_idx, train=False)
train_loader = DataLoader(train_sub, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=False, drop_last=True, prefetch_factor=2)
val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False, prefetch_factor=2)

# Mixup/CutMix and loss (lighter, prob=0.8)
mixup_fn = Mixup(
    mixup_alpha=0.2,
    cutmix_alpha=0.3,
    prob=0.8,
    switch_prob=0.5,
    label_smoothing=0.1,
    num_classes=num_classes
)
use_mixup = True
criterion = SoftTargetCrossEntropy() if use_mixup else nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = torch.amp.GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

# EMA with timm
ema = ModelEmaV2(model, decay=0.9998)

def evaluate(m, loader):
    m.eval()
    correct = 0
    total = 0
    with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            logits = m(xb)
            preds = logits.argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return correct / max(1,total)

best_acc = 0.0
best_path = 'effv2m_320_ema.pth'
t0 = time.time()
for epoch in range(epochs):
    for pg in optimizer.param_groups:
        pg['lr'] = lr_at_epoch(epoch)
    model.train()
    ep_start = time.time()
    seen = 0
    for it, (xb, yb) in enumerate(train_loader):
        xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
            if use_mixup:
                mx, my = mixup_fn(xb, yb)
                logits = model(mx)
                loss = criterion(logits, my)
            else:
                logits = model(xb)
                loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        ema.update(model)
        seen += yb.size(0)
        if (it+1) % 100 == 0:
            print(f"Ep {epoch+1}/{epochs} it {it+1} seen {seen} loss {loss.item():.4f} lr {optimizer.param_groups[0]['lr']:.2e} elapsed {time.time()-ep_start:.1f}s", flush=True)
    # Evaluate EMA model
    val_acc = evaluate(ema.module, val_loader)
    print(f"Epoch {epoch+1} val_acc(EMA) {val_acc:.4f} ep_time {time.time()-ep_start:.1f}s total {time.time()-t0:.1f}s", flush=True)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'model': ema.module.state_dict(), 'num_classes': num_classes}, best_path)
        print(f"Saved EMA best to {best_path}")

print(f"Training done. Best EMA val_acc {best_acc:.4f}")

# Inference with 2x TTA (orig + hflip) using EMA weights
print('Preparing test loader...')
with open('test2019.json','r') as f: test_json = json.load(f)
test_images = test_json['images']
test_rel = [im['file_name'] for im in test_images]
test_ids = [im['id'] for im in test_images]

class TestDS(torch.utils.data.Dataset):
    def __init__(self, rels):
        self.rels = rels
        self.tf = val_tf
    def __len__(self): return len(self.rels)
    def __getitem__(self, i):
        from PIL import Image
        p = resolve_path(self.rels[i])
        with Image.open(p) as im:
            im = im.convert('RGB')
            x = self.tf(im)
        return x

test_loader = DataLoader(TestDS(test_rel), batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False, prefetch_factor=2)

# Load EMA weights
if os.path.exists(best_path):
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt['model'], strict=False)
else:
    print('Warning: best EMA checkpoint not found; using current weights')

model.eval()
preds = []
with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
    for it, xb in enumerate(test_loader):
        xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        logits = model(xb)
        logits_flip = model(torch.flip(xb, dims=[-1]))
        logits = (logits + logits_flip) * 0.5
        preds.extend(logits.argmax(1).cpu().tolist())
        if (it+1) % 100 == 0:
            total_it = math.ceil(len(test_rel)/batch_size)
            print(f"Infer it {it+1}/{total_it}", flush=True)

assert len(preds) == len(test_ids)
pred_cat_ids = [idx_to_catid[i] for i in preds]
import pandas as pd
sub = pd.DataFrame({'image_id': test_ids, 'category_id': pred_cat_ids})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with', len(sub), 'rows')

# Notes:
# - For full run: set epochs=35, warmup_epochs=5; keep same recipe.
# - If OOM persists at batch=48, try 40/32; only then reduce image size.

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Holdout sizes -> train: 222899, val: 10100
Building timm model...


Before to(): |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    512 B   |    512 B   |    512 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |    512 B   |    512 B   |    512 B   |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |      4 B   |      4 B   |      4 B   |      0 B   |
|--------------------------------------------------

/app/.pip-target/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


...

Infer it 300/672


Infer it 400/672


Infer it 500/672


Infer it 600/672


Wrote submission.csv with 32214 rows


In [4]:
# Full 35-epoch training: tf_efficientnetv2_m.in21k_ft_in1k @320 with EMA, Mixup/CutMix, GradClip, and 6x TTA
import os, math, time, random, json, subprocess, sys
from collections import defaultdict
import torch, torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as T

os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'expandable_segments:True')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
if hasattr(torch, 'set_float32_matmul_precision'):
    torch.set_float32_matmul_precision('medium')

# AMP dtype selection (BF16 for A10); GradScaler only if FP16
cap = torch.cuda.get_device_capability(0)[0] if torch.cuda.is_available() else 0
autocast_dtype = torch.bfloat16 if (torch.cuda.is_available() and cap >= 8) else torch.float16
scaler = torch.amp.GradScaler(enabled=(autocast_dtype == torch.float16))

# Pre-reqs from earlier cells
assert 'train_ds' in globals() and 'resolve_path' in globals() and 'idx_to_catid' in globals()

# Build stratified holdout: safer cap by percentage and availability
labels = [y for _, y in train_ds.recs]
by_cls = defaultdict(list)
for i, y in enumerate(labels):
    by_cls[y].append(i)
val_idx, tr_idx = [], []
random.seed(42)
for c, idxs in by_cls.items():
    random.shuffle(idxs)
    n = len(idxs)
    v_count = min(20, max(5, n // 10))
    v_count = min(v_count, n - 1)  # leave at least one for train
    v = idxs[:v_count]
    t = idxs[v_count:]
    val_idx.extend(v)
    tr_idx.extend(t)
print(f"Holdout sizes -> train: {len(tr_idx)}, val: {len(val_idx)}", flush=True)

# Transforms
train_tf = T.Compose([
    T.RandomResizedCrop(320, scale=(0.2, 1.0), ratio=(0.75, 1.33), interpolation=T.InterpolationMode.BICUBIC),
    T.RandAugment(num_ops=2, magnitude=9),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    T.RandomErasing(p=0.10, value='random')
])
val_tf = T.Compose([
    T.Resize(352, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(320),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

class WrappedDS(torch.utils.data.Dataset):
    def __init__(self, base, indices, train):
        self.base = base
        self.idxs = indices
        self.train = train
    def __len__(self): return len(self.idxs)
    def __getitem__(self, k):
        rel, y = self.base.recs[self.idxs[k]]
        from PIL import Image
        p = resolve_path(rel)
        with Image.open(p) as img:
            img = img.convert('RGB')
            x = (train_tf if self.train else val_tf)(img)
        return x, y

# Import timm and helpers
try:
    import timm
except Exception:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'timm==1.0.9', '--no-deps'], check=True)
    import timm
from timm.loss import SoftTargetCrossEntropy
from timm.data.mixup import Mixup
from timm.utils import ModelEmaV2

num_classes = len(idx_to_catid)
print('Building model...', flush=True)
model = None
try:
    model = timm.create_model(
        'tf_efficientnetv2_m.in21k_ft_in1k',
        pretrained=True,
        num_classes=num_classes,
        drop_path_rate=0.3,
        pretrained_cfg_overlay={'hf_hub_id': None}
    )
except Exception as e:
    print('Pretrained URL fallback failed, retrying default:', e, flush=True)
    model = timm.create_model('tf_efficientnetv2_m.in21k_ft_in1k', pretrained=True, num_classes=num_classes, drop_path_rate=0.3)

if hasattr(model, 'set_grad_checkpointing'):
    model.set_grad_checkpointing(True)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
model = model.to(device)
model = model.to(memory_format=torch.channels_last)

# Disable torch.compile due to Triton requiring Python.h in this env; use eager for stability
try:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
except Exception:
    pass
print("Skipping torch.compile: using eager mode.", flush=True)

# Optimizer, sched, EMA, loss, mixup
init_batch = 64
batch_size = init_batch
base_lr = 3e-4 * (batch_size/64)
optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=0.05, betas=(0.9,0.999))
epochs = 35
warmup_epochs = 5
cosine_min_lr = 1e-6

def lr_at_epoch(ep):
    if ep < warmup_epochs:
        return base_lr * (ep+1)/max(1,warmup_epochs)
    t = (ep - warmup_epochs)/max(1,(epochs - warmup_epochs))
    return cosine_min_lr + 0.5*(base_lr - cosine_min_lr)*(1 + math.cos(math.pi*t))

mixup_fn = Mixup(
    mixup_alpha=0.2,
    cutmix_alpha=0.3,
    prob=0.7,
    switch_prob=0.5,
    label_smoothing=0.1,
    num_classes=num_classes
)
use_mixup = True
criterion = SoftTargetCrossEntropy() if use_mixup else nn.CrossEntropyLoss(label_smoothing=0.1)
ema = ModelEmaV2(model, decay=0.9995)  # ramp to 0.9998

# DataLoaders (after model for memory safety), tuned throughput
num_workers = min(12, os.cpu_count() or 12)
train_sub = WrappedDS(train_ds, tr_idx, train=True)
val_sub = WrappedDS(train_ds, val_idx, train=False)

def make_loaders(bs):
    tl = DataLoader(train_sub, batch_size=bs, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=False, drop_last=True, prefetch_factor=4)
    vl = DataLoader(val_sub, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False, prefetch_factor=4)
    return tl, vl

# Try larger batch, fallback on OOM automatically with quick forward sanity
for bs_try in (64, 56, 48, 40, 32):
    try:
        torch.cuda.empty_cache()
        train_loader, val_loader = make_loaders(bs_try)
        xb, yb = next(iter(train_loader))
        with torch.no_grad(), torch.amp.autocast(device_type='cuda', dtype=autocast_dtype, enabled=torch.cuda.is_available()):
            _ = model(xb.to(device, non_blocking=True).to(memory_format=torch.channels_last))
        batch_size = bs_try
        print(f"Using batch_size={batch_size}", flush=True)
        break
    except RuntimeError as e:
        if 'out of memory' in str(e).lower():
            print(f"OOM with batch_size={bs_try}, reducing...", flush=True)
            try:
                del train_loader, val_loader
            except Exception:
                pass
            torch.cuda.empty_cache()
            continue
        else:
            raise

# Update LR to reflect final batch size
for pg in optimizer.param_groups:
    pg['lr'] = 3e-4 * (batch_size/64)
base_lr = optimizer.param_groups[0]['lr']
print(f"Base LR set to {base_lr:.2e}", flush=True)

def evaluate(m, loader):
    m.eval()
    correct = 0
    total = 0
    with torch.no_grad(), torch.amp.autocast(device_type='cuda', dtype=autocast_dtype, enabled=torch.cuda.is_available()):
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            yb = yb.to(device, non_blocking=True)
            logits = m(xb)
            preds = logits.argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return correct / max(1,total)

best_acc = 0.0
best_path = 'effv2m_320_ema.pth'
t0 = time.time()
update_count = 0
for epoch in range(epochs):
    # Set LR for epoch
    cur_lr = lr_at_epoch(epoch)
    for pg in optimizer.param_groups:
        pg['lr'] = cur_lr
    # Ramp EMA decay from 0.9995 -> 0.9998 across training
    ema.decay = 0.9995 + (0.9998 - 0.9995) * (epoch / max(1, epochs-1))
    model.train()
    ep_start = time.time()
    seen = 0
    for it, (xb, yb) in enumerate(train_loader):
        xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type='cuda', dtype=autocast_dtype, enabled=torch.cuda.is_available()):
            if use_mixup:
                mx, my = mixup_fn(xb, yb)
                logits = model(mx)
                loss = criterion(logits, my)
            else:
                logits = model(xb)
                loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        # Grad clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        ema.update(model)
        update_count += 1
        seen += yb.size(0)
        if (it+1) % 200 == 0:
            print(f"Ep {epoch+1}/{epochs} it {it+1} seen {seen} loss {loss.item():.4f} lr {cur_lr:.2e} ema {ema.decay:.6f} elapsed {time.time()-ep_start:.1f}s", flush=True)
    val_acc = evaluate(ema.module, val_loader)
    print(f"Epoch {epoch+1} val_acc(EMA) {val_acc:.4f} ep_time {time.time()-ep_start:.1f}s total {time.time()-t0:.1f}s", flush=True)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({'model': ema.module.state_dict(), 'num_classes': num_classes}, best_path)
        print(f"Saved EMA best to {best_path}", flush=True)
    # Save 'last' checkpoint every epoch for resume
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'ema_state_dict': ema.state_dict(),
        'scaler_state_dict': getattr(scaler, 'state_dict', lambda: {})(),
        'best_acc': best_acc,
        'batch_size': batch_size,
    }, 'checkpoint-last.pth')
    # Periodic EMA snapshot every 5 epochs
    if (epoch + 1) % 5 == 0:
        p = f'effv2m_320_ema_ep{epoch+1}.pth'
        torch.save({'model': ema.module.state_dict(), 'num_classes': num_classes}, p)
        print(f"Saved periodic EMA to {p}", flush=True)

print(f"Training done. Best EMA val_acc {best_acc:.4f}", flush=True)

# Inference with 6x TTA: scales [288,320,352] x {orig, hflip}, CPU accumulation and 320 weighted 2x
with open('test2019.json','r') as f: test_json = json.load(f)
test_images = test_json['images']
test_rel = [im['file_name'] for im in test_images]
test_ids = [im['id'] for im in test_images]

def make_val_tf(scale):
    resize_side = scale + 32
    return T.Compose([
        T.Resize(resize_side, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(scale),
        T.ToTensor(),
        T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])

class TestDS(torch.utils.data.Dataset):
    def __init__(self, rels, tf):
        self.rels = rels
        self.tf = tf
    def __len__(self): return len(self.rels)
    def __getitem__(self, i):
        from PIL import Image
        p = resolve_path(self.rels[i])
        with Image.open(p) as im:
            im = im.convert('RGB')
            x = self.tf(im)
        return x

# Load EMA weights
ckpt = torch.load(best_path, map_location=device)
model.load_state_dict(ckpt['model'], strict=False)
model.eval()

scales = [288, 320, 352]
tta_logits = torch.zeros((len(test_rel), num_classes), dtype=torch.float32)  # CPU accumulation
tta_weight = 0.0
num_workers = min(12, os.cpu_count() or 12)

with torch.no_grad():
    for s in scales:
        tf_s = make_val_tf(s)
        ds_s = TestDS(test_rel, tf_s)
        dl_s = DataLoader(ds_s, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False, prefetch_factor=4)
        w = 2.0 if s == 320 else 1.0
        idx = 0
        for it, xb in enumerate(dl_s):
            xb = xb.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast(device_type='cuda', dtype=autocast_dtype, enabled=torch.cuda.is_available()):
                lg = model(xb)
                lg_flip = model(torch.flip(xb, dims=[-1]))
                lg = (lg + lg_flip) * 0.5
            lg = lg.float().cpu()
            bsz = lg.shape[0]
            tta_logits[idx:idx+bsz] += lg * w
            idx += bsz
            if (it+1) % 100 == 0:
                total_it = math.ceil(len(ds_s)/batch_size)
                print(f"TTA scale {s} it {it+1}/{total_it}", flush=True)
        tta_weight += w

tta_logits /= max(1.0, tta_weight)
preds = tta_logits.argmax(1).tolist()
pred_cat_ids = [idx_to_catid[i] for i in preds]
import pandas as pd
sub = pd.DataFrame({'image_id': test_ids, 'category_id': pred_cat_ids})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with', len(sub), 'rows')
print('Done.')

Holdout sizes -> train: 217930, val: 15069


Building model...


Skipping torch.compile: using eager mode.


/app/.pip-target/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/app/.pip-target/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Using batch_size=64


...

TTA scale 352 it 200/504


TTA scale 352 it 300/504


TTA scale 352 it 400/504


TTA scale 352 it 500/504


Wrote submission.csv with 32214 rows
Done.


In [6]:
# Diagnose and fix submission format to match Kaggle sample
import pandas as pd
import os

sample_path = 'kaggle_sample_submission.csv'
sub_path = 'submission.csv'
assert os.path.exists(sample_path), 'kaggle_sample_submission.csv not found'
assert os.path.exists(sub_path), 'submission.csv not found'

sample = pd.read_csv(sample_path)
sub = pd.read_csv(sub_path)
print('Sample columns:', list(sample.columns))
print('Submission columns:', list(sub.columns))
print('Sample shape:', sample.shape, 'Submission shape:', sub.shape)

# Ensure columns and order match sample exactly
required_cols = list(sample.columns)  # expected e.g. ['id', 'predicted']
rename_map = {}
# Map common alt names to required ones
if 'id' in required_cols and 'id' not in sub.columns and 'image_id' in sub.columns:
    rename_map['image_id'] = 'id'
if 'predicted' in required_cols and 'predicted' not in sub.columns and 'category_id' in sub.columns:
    rename_map['category_id'] = 'predicted'
if rename_map:
    sub = sub.rename(columns=rename_map)

# Reorder and filter to required columns only
missing = [c for c in required_cols if c not in sub.columns]
assert not missing, f'Missing columns in submission: {missing}'
sub = sub[required_cols]

# Enforce integer dtypes for id/predicted
for c in required_cols:
    if pd.api.types.is_float_dtype(sub[c]):
        sub[c] = sub[c].round().astype('int64')
    elif not pd.api.types.is_integer_dtype(sub[c]):
        try:
            sub[c] = sub[c].astype('int64')
        except Exception:
            pass

# Final sanity
assert sub.shape[0] == sample.shape[0], f'Row count mismatch: {sub.shape[0]} vs sample {sample.shape[0]}'
assert list(sub.columns) == required_cols, f'Column order mismatch: {list(sub.columns)} vs {required_cols}'

sub.to_csv(sub_path, index=False)
print('Fixed submission saved. Columns:', list(sub.columns), 'shape:', sub.shape)

Sample columns: ['id', 'predicted']
Submission columns: ['image_id', 'category_id']
Sample shape: (32214, 2) Submission shape: (32214, 2)
Fixed submission saved. Columns: ['id', 'predicted'] shape: (32214, 2)
